<div style="background-image: url('https://cdn-images-1.medium.com/max/1200/1*OOWSoWHeQ5kyJ4N0P2ptNA.png');">
<p style="text-align: center;">
  <a href="https://itb.ac.id/">
    <img src="https://ditsti.itb.ac.id/wp-content/uploads/2020/11/logo_itb_1024_bw.png" alt="Logo" width=72 height=72>
  </a>

  <h3 style="text-align: center;">IF3270 Machine Learning</h3>

  <p style="text-align: center;">
    Mengimplementasikan Algoritma FFNN
    <br style="text-align: center;">
    <br style="text-align: center;">
    <!-- <a href="https://reponame/issues/new?template=bug.md">Developer</a>
    ·
    <a href="https://reponame/issues/new?template=feature.md&labels=feature">Table of Contents</a> -->
  </p>
</p>
</div>

In [247]:
!pip install -r requirements.txt

In [248]:
# import library
import json
import math
import numpy as np

In [249]:
# read model file from filesystem
f = open("model1.json", "r")

# loads model and parse to json object
model = json.loads(f.read())

network_depth = model["Network_Depth"]
learning_rate = model["Learning_Rate"]
num_of_input = model["Num_Of_Input"]
layer_group = model["Layers"]
weight_group = model["Weights"]
input_group = model["Input"]

In [250]:
class Layer:
    def __init__(self, width, activation=None):
        '''
        [Attributes]
            _width      : int       { banyak node pada layer }
            _activation : func      { fungsi aktivasi layer, None jika layer input }
            nodes       : list[float] / list[int] { nilai setiap node pada layer }
        '''
        self._width = width
       
        # Set the activation function self._activation = None
        self._activation = self.linear
        if activation is not None:
            if activation == "linear":
                self._activation = self.linear
            if activation == "sigmoid":
                self._activation = self.sigmoid
            if activation == "relu":
                self._activation = self.relu
            if activation == "softmax":
                self._activation = self.softmax
            
        self.nodes = [0] * self._width
    
    def set_nodes(self, val=0):
        '''
        [DESC]
            Menginisialisasi nilai setiap node pada layer
        [PARAMS]
            val     : int/float         { nilai yang akan diinisialisasi untuk setiap node pada layer }
            val     : list[int]|list[float]
        '''
        if isinstance(val, (int, float)):
            self.nodes = [val] * self._width
        if isinstance(val, list):
            self.nodes = val
    
    '''Activation Functions'''
    def activate(self):
        '''
        [DESC]
            Abstraksi yang dipanggil di NeuralNetwork untuk melakukan
            fungsi aktivasi pada setiap node di layer ini
        '''
        self._activation()

    def sigmoid(self):
        '''
        [DESC]
            Mengubah attr nodes dari perkalian vektor weight dengan node
            menjadi list dengan rentang nilai 0 - 1
        [PARAMS]
            
        '''
        for i, z in enumerate(self.nodes):
            self.nodes[i] = 1 / (1 + math.exp(-z))
    
    def linear(self):
        '''
        [DESC]
            Mengubah attr nodes dari perkalian vektor weight dengan node
        [PARAMS]
            X   : list[float]     { input node }
            W   : list[float]     { weight setiap node }
        '''
        pass

    def relu(self):
        '''
        [DESC]
            Menghitung hasil fungsi aktivasi relu.
            Jika nilai dibawah 0 maka kembalian adalah 0, Jika tidak maka mengembalikan nilai dari fungsi linear.
        [PARAMS]
            X   : nilai dari masukan dalam bentuk array of array (Matrix)
            W   : weight dalam bentuk array of array (Matrix)
        '''
        for i, z in enumerate(self.nodes):
            self.nodes[i] = max(0, z)
    
    def softmax(self):
        '''
        [DESC]
            Mengubah attr nodes dengan nilai perbandingan seluruh node
        '''
        temp_z_exps = [np.exp(zi) for zi in self.nodes] # calculate exp(z) for all node in layer
        self.sum_exp_z = np.sum(temp_z_exps)
        for i, z in enumerate(temp_z_exps):
            self.nodes[i] = z / self.sum_exp_z

In [251]:
class NeuralNetwork:
    def __init__(self, data):
        '''
        [ATTRIBUTES]
            n_
        Nilai data diisi dari model
        '''
        self.depth = data["Network_Depth"]
        self.n_input = data["Num_Of_Input"]
        
        self._layers = []
        for i, layer in enumerate(data["Layers"]):
            assert i == layer["depth"]  # check that the json file did not miss a layer
            n_node = layer["width"]
            activation = layer["activation"]
            temp_layer = Layer(width=n_node, activation=activation)
            self._layers.append(temp_layer)
        
        self._weights = [0] * len(data["Weights"])
        for i, weight in enumerate(data["Weights"]):
            assert i == weight["depth_origin"]
            self._weights[i] = weight["values"]

    def init_input_layer(self, input_instance):
        '''
        [DESC]
            Menginisialisasi input layer dengan instance dari dataset
        [PARAMS]
            input_instance      : list[int|float]   { sebuah input dari dataset }
        '''
        self._layers[0].nodes = input_instance
    
    def feed_forward(self, input_group):
        '''
        [DESC]
            Melakukan proses feed forward
        [PARAMS]
            input       : input dari file hmm atau dalam bentuk list atau Layer(?)
        [RETURN]
            output      : Layer     { layer output } \t(1)
            output      : list[int|float]     { list }     (2)
            # belum diputuskan yg mana 1/2
        '''
        hasil = [0] * self.n_input
        for i, masukan in enumerate(input_group):
            masukan_temp = [1] + masukan
            self.init_input_layer(masukan)
            for j in range(1, network_depth):
                layer = self._layers[j]
                w = np.array(self._weights[j-1])    # matrix of weights
                x = np.array(masukan_temp)          # array of input from lower layer
                sigma = np.matmul(w,x)              # matrix multiplication
                layer.nodes = sigma.tolist()        # assignment from result multiplication into array nodes
                
                layer.activate()
                    
                '''hasil aktivasi dijadikan sebagai input pada layer di atasnya
                perlu ditambahkan '1' sebagai bias untuk hidden layer'''
                if(j == network_depth-1):
                    masukan_temp = layer.nodes
                    break
                else:
                    temp = [0] * (layer._width+1)
                    temp[0] = 1
                    for w in range(layer._width):
                        temp[w+1] = layer.nodes[w]
                    masukan_temp = temp
                
            hasil[i] = masukan_temp
            print(f'hasil data {i+1}: {masukan} -> {[round(e) for e in hasil[i]]}')
                

IndentationError: expected an indented block after function definition on line 72 (829060201.py, line 74)

In [ ]:
neural_network = NeuralNetwork(model)
neural_network.feed_forward(input_group)

hasil loop 0: [0, 0] -> [0]
hasil loop 1: [0, 1] -> [1]
hasil loop 2: [1, 0] -> [1]
hasil loop 3: [1, 1] -> [0]


## Visualization

In [ ]:
from graphviz import Source

In [ ]:
def build_format(model: NeuralNetwork):
    '''
    [DESC]
        Buat visualisasi untuk model dalam format graphviz
    [PARAMS]
        model       : NeuralNetwork     { model yang akan divisualisasikan }
    '''

    from keras.models import Sequential
    from keras.layers import Dense

    # contoh dari internet
    graph = temp = '''
    digraph G {
        graph[ fontname = "Helvetica-Oblique",
                fontsize = 12,
                label = "",
                size = "7.75,10.25" ];
    
        rankdir = LR;
        splines=false;
        edge[style=invis];
        ranksep= 1.4;
        {
            node [shape=circle, color=chartreuse, style=filled, fillcolor=chartreuse];
            x1 [label=<x1>];
            x2 [label=<x2>]; 
        }
        {
            node [shape=circle, color=dodgerblue, style=filled, fillcolor=dodgerblue];
            a12 [label=<a<sub>1</sub><sup>(2)</sup>>];
            a22 [label=<a<sub>2</sub><sup>(2)</sup>>];
            a32 [label=<a<sub>3</sub><sup>(2)</sup>>];
            a42 [label=<a<sub>4</sub><sup>(2)</sup>>];
            a52 [label=<a<sub>5</sub><sup>(2)</sup>>];
            a13 [label=<a<sub>1</sub><sup>(3)</sup>>];
            a23 [label=<a<sub>2</sub><sup>(3)</sup>>];
            a33 [label=<a<sub>3</sub><sup>(3)</sup>>];
            a43 [label=<a<sub>4</sub><sup>(3)</sup>>];
            a53 [label=<a<sub>5</sub><sup>(3)</sup>>];
        }
        {
            node [shape=circle, color=coral1, style=filled, fillcolor=coral1];
            O1 [label=<y1>];
            O2 [label=<y2>]; 
            O3 [label=<y3>]; 
        }
        {
            rank=same;
            x1->x2;
        }
        {
            rank=same;
            a12->a22->a32->a42->a52;
        }
        {
            rank=same;
            a13->a23->a33->a43->a53;
        }
        {
            rank=same;
            O1->O2->O3;
        }
        l0 [shape=plaintext, label="layer 1 (input layer)"];
        l0->x1;
        {rank=same; l0;x1};
        l1 [shape=plaintext, label="layer 2 (hidden layer)"];
        l1->a12;
        {rank=same; l1;a12};
        l2 [shape=plaintext, label="layer 3 (hidden layer)"];
        l2->a13;
        {rank=same; l2;a13};
        l3 [shape=plaintext, label="layer 4 (output layer)"];
        l3->O1;
        {rank=same; l3;O1};
        edge[style=solid, tailport=e, headport=w];
        {x1; x2} -> {a12;a22;a32;a42;a52};
        {a12;a22;a32;a42;a52} -> {a13;a23;a33;a43;a53};
        {a13;a23;a33;a43;a53} -> {O1,O2,O3};
    }'''
    
from graphviz import Source
dot = Source(graph)
dot.format = 'png'
dot.render('neural_network_01', view=False) 
'neural_network_01.png'

![](https://moonbooks.org/media/images/src/neural-network-01.PNG)